In [1]:
import cv2
import os
import traceback
import mediapipe as mp
import numpy as np
import pandas as pd
import IPython.display as ipd
from PIL import Image, ImageDraw, ImageFont
from sympy import Point, Polygon 
from tqdm.notebook import tqdm
import time
import json
import math
import msgpack
import requests
from threading import Thread
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from typing import List
import uvicorn
import httpx
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def exercitiu_live(tip_exercitiu):
    #Pentru Vid live
    #Feed-ul de la camera
    global pose_model
    global puncte_linii_suport_dr
    global puncte_linii_suport_st
    global mana_dreapta
    global puncte
    global puncte_metrii
    global stare
    global cnt_genuflexiuni_front
    global cnt_genuflexiuni_profil
    global moment_mana_ridicata
    global coordonate_genuflexiuni_front
    global coordonate_genuflexiuni_profil
    global etapa_anterioara
    global distanta_scalare

    cnt_genuflexiuni_front = 0
    cnt_genuflexiuni_profil = 0
    stare = 0
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_HEEL, mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER]
    mana_dreapta = mp_pose.PoseLandmark.RIGHT_INDEX
    moment_mana_ridicata = None
    coordonate_genuflexiuni_front = [{"coborare": [], "urcare": []} for _ in range(5)]
    coordonate_genuflexiuni_profil = [{"coborare": [], "urcare": []} for _ in range(5)]
    etapa_anterioara = None
    distanta_scalare = None
    distanta_scalare_profil = None

    
    cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)
    
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5, model_complexity=2) as pose:
        while cam.isOpened():
            ret, frame = cam.read()
    
            #colorarea frame-ului din BGR in RGB
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False
    
            #detectarea modelului
            pose_model = pose.process(img)
            try:
                puncte = pose_model.pose_landmarks.landmark
                puncte_metrii = pose_model.pose_world_landmarks.landmark
                
            except Exception as e:
                print(f"Eroare de detectie a modelului: {e}")
            
            #colorarea frame-ului din RGB in BGR
            img.flags.writeable = True
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
            mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
           

            
            img = analiza_fandare_front(img)
                
            cv2.imshow("Camera Feed", img)
            
            cv2.waitKey(1)
            if cv2.getWindowProperty("Camera Feed", cv2.WND_PROP_VISIBLE) < 1:
                break
        cam.release()
        cv2.destroyAllWindows()
        calcul_adancime_genuflexiune(coordonate_genuflexiuni_front, distanta_scalare)
        calcul_hip_shift_front(coordonate_genuflexiuni_front, distanta_scalare)

        calcul_adancime_genuflexiune(coordonate_genuflexiuni_profil, distanta_scalare)
        calcul_unghiuri_profil(coordonate_genuflexiuni_profil)

In [3]:
def exercitiu_live1():
    global pose_model
    global puncte_linii_suport_dr
    global puncte_linii_suport_st
    global mana_dreapta
    global puncte
    global puncte_metrii
    global cnt_genuflexiuni_front
    global cnt_genuflexiuni_profil
    global moment_mana_ridicata
    global coordonate_genuflexiuni_front
    global coordonate_genuflexiuni_profil
    global etapa_anterioara
    global distanta_scalare

    cnt_genuflexiuni_front = 0
    cnt_genuflexiuni_profil = 0
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_HEEL, mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER]
    mana_dreapta = mp_pose.PoseLandmark.RIGHT_INDEX
    moment_mana_ridicata = None
    coordonate_genuflexiuni_front = [{"coborare": [], "urcare": []} for _ in range(5)]
    coordonate_genuflexiuni_profil = [{"coborare": [], "urcare": []} for _ in range(5)]
    etapa_anterioara = None
    distanta_scalare = None
    distanta_scalare_profil = None

    prelucrare_stream()



In [4]:
def prelucrare_stream():
    
    

    app.add_middleware(
        CORSMiddleware,
        allow_origins = ["*"],  # Permite cereri din orice sursă
        allow_credentials = True,
        allow_methods = ["*"],  # Permite toate metodele HTTP (GET, POST etc.)
        allow_headers= ["*"],  # Permite toate header-ele
    )
    @app.websocket("/video_feed_live/")
    async def video_feed_live(websocket_genuflexiuni : WebSocket):

       

        global pose_model
        global puncte_linii_suport_dr
        global puncte_linii_suport_st
        global mana_dreapta
        global puncte
        global puncte_metrii
        global cnt_genuflexiuni_front
        global cnt_genuflexiuni_profil
        global moment_mana_ridicata
        global coordonate_genuflexiuni_front
        global coordonate_genuflexiuni_profil
        global etapa_anterioara
        global distanta_scalare
        global stare
    
        cnt_genuflexiuni_front = 0
        cnt_genuflexiuni_profil = 0
        puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_HEEL, mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER]
        puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER]
        mana_dreapta = mp_pose.PoseLandmark.RIGHT_INDEX
        moment_mana_ridicata = None
        coordonate_genuflexiuni_front = [{"coborare": [], "urcare": []} for _ in range(5)]
        coordonate_genuflexiuni_profil = [{"coborare": [], "urcare": []} for _ in range(5)]
        etapa_anterioara = None
        distanta_scalare = None
        distanta_scalare_profil = None
        
        await websocket_genuflexiuni.accept()
        cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        exercitiu_in_curs = 1
        stare = 0
        with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5, model_complexity=2) as pose:
            while exercitiu_in_curs:
                ret, frame = cam.read()
    
                if not ret:
                    break
                    
                #colorarea frame-ului din BGR in RGB
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img.flags.writeable = False
        
                #detectarea modelului
                pose_model = pose.process(img)
                try:
                    puncte = pose_model.pose_landmarks.landmark
                    puncte_metrii = pose_model.pose_world_landmarks.landmark
                    
                except Exception as e:
                    print(f"Eroare de detectie a modelului: {e}")
                
                #colorarea frame-ului din RGB in BGR
                img.flags.writeable = True
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
                mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
                img, exercitiu_in_curs = analiza_fandare_front(img, puncte, puncte_metrii)
                
                _, buffer = cv2.imencode('.jpg', img)
    
                await websocket_genuflexiuni.send_bytes(buffer.tobytes())

        cam.release()
        await websocket_genuflexiuni.close()
        
        adancimi_front = calcul_adancime_genuflexiune(coordonate_genuflexiuni_front, distanta_scalare)
        medii_hip_shift_frontal, medii_diferenta_nivel_solduri, medii_diferenta_nivel_solduri_z = calcul_hip_shift_front(coordonate_genuflexiuni_front, distanta_scalare)
    
        adancimi_profil = calcul_adancime_genuflexiune(coordonate_genuflexiuni_profil, distanta_scalare)
        unghiuri_profil = calcul_unghiuri_profil(coordonate_genuflexiuni_profil)

        
        date_extrase_front, _ =  preproc_date_json(adancimi_front, medii_hip_shift_frontal, medii_diferenta_nivel_solduri, medii_diferenta_nivel_solduri_z, adancimi_profil, unghiuri_profil)
        #pachet_date_front = {"data": date_extrase_front}
        try:
            async with httpx.AsyncClient() as client:
                resp = await client.post("http://127.0.0.1:8000/date_extrase_genuflexiuni/", json = date_extrase_front)
        except Exception as e:
            print("Nu s-au putut trimite datele despre genuflexiuni la API: ", e)
            

In [5]:
def preproc_date_json(adancimi_front, medii_hip_shift_frontal, medii_diferenta_nivel_solduri, medii_diferenta_nivel_solduri_z, adancimi_profil, unghiuri_profil):
    date_genuflexiuni_front = []
    date_genuflexiuni_profil = []

    for i in range(5):
        dict_genuflexiuni_front = {}
        dict_genuflexiuni_profil = {}

        dict_genuflexiuni_front["adancime"] = adancimi_front[i]
        dict_genuflexiuni_front["deviatie_coborare"] = medii_hip_shift_frontal[i]["coborare"]
        dict_genuflexiuni_front["deviatie_urcare"] = medii_hip_shift_frontal[i]["urcare"]
        dict_genuflexiuni_front["dif_nivel_solduri_coborare"] = medii_diferenta_nivel_solduri[i]["coborare"]
        dict_genuflexiuni_front["dif_nivel_solduri_urcare"] = medii_diferenta_nivel_solduri[i]["urcare"]
        dict_genuflexiuni_front["dif_nivel_solduri_z_coborare"] = medii_diferenta_nivel_solduri_z[i]["coborare"]
        dict_genuflexiuni_front["dif_nivel_solduri_z_urcare"] = medii_diferenta_nivel_solduri_z[i]["urcare"]
        
        dict_genuflexiuni_profil["adancime"] = adancimi_profil[i]
        dict_genuflexiuni_profil["flexie_sold"] = unghiuri_profil["FLEXIE_SOLD"][i]
        dict_genuflexiuni_profil["flexie_genunchi"] = unghiuri_profil["FLEXIE_GENUNCHI"][i]
        dict_genuflexiuni_profil["dorisflexie_glezna"] = unghiuri_profil["UNGHI_TIBIE"][i]

        date_genuflexiuni_front.append(dict_genuflexiuni_front)
        date_genuflexiuni_profil.append(dict_genuflexiuni_profil)

    print(date_genuflexiuni_front)
    print(date_genuflexiuni_profil)
        
    return date_genuflexiuni_front, date_genuflexiuni_profil
    

In [6]:
def exercitiu_live_prototip(tip_exercitiu): #Pentru midel cu Task-uri, practic mai nou, dar nu gaseste absolute path pt model_path de niciun fel
    #Pentru Vid live
    #Feed-ul de la camera
    global pose_model
    global puncte_linii_suport_dr
    global puncte_linii_suport_st
    global mana_dreapta
    global nas
    global puncte
    global stare
    global cnt_genuflexiuni_front
    global cnt_genuflexiuni_profil
    global moment_mana_ridicata
    global coordonate_genuflexiuni_front
    global coordonate_genuflexiuni_profil
    global etapa_anterioara

    model_path = r"pose_landmarker_full.task"
    cnt_genuflexiuni_front = 0
    cnt_genuflexiuni_profil = 0
    stare = 0
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_HEEL, mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER]
    mana_dreapta = mp_pose.PoseLandmark.RIGHT_INDEX
    moment_mana_ridicata = None
    coordonate_genuflexiuni_front = [{"coborare": [], "urcare": []} for _ in range(5)]
    coordonate_genuflexiuni_profil = [{"coborare": [], "urcare": []} for _ in range(5)]
    etapa_anterioara = None
    cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)
    
    BaseOptions = mp.tasks.BaseOptions
    PoseLandmarker = mp.tasks.vision.PoseLandmarker
    PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode

    options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

    with PoseLandmarker.create_from_options(options) as landmarker:
        while cam.isOpened():
            ret, frame = cam.read()
    
            #colorarea frame-ului din BGR in RGB
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False
    
            #detectarea modelului
            pose_model = landmarker.detect(img)
            try:
                 if pose_result.pose_landmarks:
                    for landmark in pose_result.pose_landmarks[0]:  # Luăm prima persoană detectată
                        x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            except Exception as e:
                print(f"Eroare de detectie a modelului: {e}")
            
            #colorarea frame-ului din RGB in BGR
            img.flags.writeable = True
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
            #mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
           

            
            img = analiza_fandare_front(img)
                
            cv2.imshow("Camera Feed", img)
            
            cv2.waitKey(1)
            if cv2.getWindowProperty("Camera Feed", cv2.WND_PROP_VISIBLE) < 1:
                break
        cam.release()
        cv2.destroyAllWindows()

In [7]:
def adaug_text(img, indicatii):
    try:
        dif = 5
        for indicatie in indicatii:
            img = cv2.putText(img, indicatie, (5, img.shape[0] - dif), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            dif += 40
    except Exception as e:
        print(f"Eroare la adaug_text: {e}")
    return img

In [8]:
def analiza_fandare_front(img, puncte, puncte_metrii):
    coord_pct_suport_dr = []
    coord_pct_suport_dr_metrii = [] #relativ la punctul central dintre solduri
    denumire_pct_suport_dr = []
    coord_pct_suport_st = []
    coord_pct_suport_st_metrii = [] #relativ la punctul central dintre solduri
    denumire_pct_suport_st = []
    indicatii = []
    coord_y_mana_dreapta = None
    exercitiu_in_curs = 1
    
    
    for (puncte_cheie_dr, puncte_cheie_st) in zip(puncte_linii_suport_dr, puncte_linii_suport_st) :
        if puncte[puncte_cheie_dr.value].visibility > 0.5:
            coord_pct_suport_dr.append((int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]), puncte[puncte_cheie_dr.value].z))
    
        if puncte[puncte_cheie_st.value].visibility > 0.5:
            coord_pct_suport_st.append((int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]), puncte[puncte_cheie_st.value].z))
            
            #print(puncte_cheie_dr.value)
            #print(puncte[puncte_cheie_dr.value].visibility)
            #print(puncte_cheie_st.value)
            #print(puncte[puncte_cheie_st.value].visibility)
    #print(afla_orientare(puncte))
    print(puncte[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility)
    print(puncte[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility)
    print(puncte[mp_pose.PoseLandmark.RIGHT_KNEE].visibility)
    print(puncte[mp_pose.PoseLandmark.LEFT_KNEE].visibility)
    
    if puncte[mana_dreapta.value].visibility > 0.5:
        coord_y_mana_dreapta = int(puncte[mana_dreapta.value].y * img.shape[0])

    if puncte[mp_pose.PoseLandmark.NOSE].visibility > 0.5:
        coord_y_nas = (int(puncte[mp_pose.PoseLandmark.NOSE.value].x * img.shape[1]), int(puncte[mp_pose.PoseLandmark.NOSE.value].y * img.shape[0]), puncte[mp_pose.PoseLandmark.NOSE.value].z)

    if puncte[mp_pose.PoseLandmark.RIGHT_KNEE].visibility > 0.5:
        coord_y_genunchi_m = (puncte_metrii[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, puncte_metrii[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, puncte_metrii[mp_pose.PoseLandmark.RIGHT_KNEE.value].z)

    if puncte[mp_pose.PoseLandmark.RIGHT_HIP].visibility > 0.5:
        coord_sold_dr_metrii = (puncte_metrii[mp_pose.PoseLandmark.RIGHT_HIP.value].x, puncte_metrii[mp_pose.PoseLandmark.RIGHT_HIP.value].y, puncte_metrii[mp_pose.PoseLandmark.RIGHT_HIP.value].z)

    if puncte[mp_pose.PoseLandmark.LEFT_HIP].visibility > 0.5:
        coord_sold_st_metrii = (puncte_metrii[mp_pose.PoseLandmark.LEFT_HIP.value].x, puncte_metrii[mp_pose.PoseLandmark.LEFT_HIP.value].y, puncte_metrii[mp_pose.PoseLandmark.LEFT_HIP.value].z)
    
    
    if stare == 0:
        try:
            indicatii = pregatire_genuflexiune_front(coord_pct_suport_dr, coord_pct_suport_st, coord_y_mana_dreapta, coord_sold_dr_metrii)
        except:
            indicatii.append("intrati in cadru")
    if stare == 1:
        try:
            (indicatii, etapa_genuflexiune) = proces_genuflexiune_front(coord_pct_suport_dr, coord_pct_suport_st)
            stocare_date_front(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas, coord_sold_dr_metrii, coord_sold_st_metrii, etapa_genuflexiune)
        except Exception as e:
            indicatii.append("Exercitiu intrerupt!")
            print (e)
            traceback.print_exc()

    if stare == 2:
        indicatii = pregatire_genuflexiune_profil(coord_pct_suport_dr, coord_pct_suport_st, coord_y_mana_dreapta, puncte)

    if stare == 3:
        try:
            (indicatii, etapa_genuflexiune) = proces_genuflexiune_profil(coord_pct_suport_dr)
            stocare_date_profil(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas, etapa_genuflexiune)
        except Exception as e:
           indicatii.append("Exercitiu intrerupt!")
           print(e)
           traceback.print_exc()

    if stare == 4:
        exercitiu_in_curs = 0
    #intra din 1 in 0 daca fac squat nu are sens!!!!
    
    img = adaug_text(img, indicatii)
    print(stare)
    
    return img, exercitiu_in_curs

In [9]:
def proces_genuflexiune_front(coord_pct_suport_dr, coord_pct_suport_st):
    #aici as putea introduce  calculul scorului cand fac exercitiul
    global cnt_genuflexiuni_front
    global coordonate_genuflexiuni
    global stare
    global etapa_anterioara
    indicatii = [f"Counter: {cnt_genuflexiuni_front}/5"]

    if len(coord_pct_suport_dr) == 6 and len(coord_pct_suport_st) == 6:

        
        
        flexie_genunchi_dr = calcul_unghi_3_puncte(coord_pct_suport_dr[2], coord_pct_suport_dr[3], coord_pct_suport_dr[4])
        flexie_genunchi_st = calcul_unghi_3_puncte(coord_pct_suport_st[2], coord_pct_suport_st[3], coord_pct_suport_st[4])
        
        if flexie_genunchi_dr >= 150 and flexie_genunchi_st >= 150:
            if etapa_anterioara == "urcare" and cnt_genuflexiuni_front < 5:
                cnt_genuflexiuni_front +=1
            if cnt_genuflexiuni_front == 5:
                stare = 2
                etapa_anterioara = None
                
            etapa_anterioara = "coborare"
            etapa_genuflexiune = "coborare"
            
        elif flexie_genunchi_dr < 75 and flexie_genunchi_st < 75:
            if etapa_anterioara == "coborare":
                etapa_anterioara = "urcare"
            etapa_genuflexiune = "urcare"

        else:
            etapa_genuflexiune = etapa_anterioara
            
        
        try:
            indicatii.append(etapa_genuflexiune)
        except Exception as e:
            print(e)
        print(flexie_genunchi_dr)
        print(flexie_genunchi_st)
        print (etapa_anterioara)
    else:
        indicatii.append("intrati in cadru")
        
    return indicatii, etapa_genuflexiune
    
    

In [10]:
def pregatire_genuflexiune_front(coord_pct_suport_dr, coord_pct_suport_st, coord_y_mana_dreapta, coord_sold_dr_metri):

    global stare
    global moment_mana_ridicata
    global distanta_scalare
    
    indicatii = []
    unghi_talpi = False
    pozitionare_picioare = False
    
    if len(coord_pct_suport_dr) == 6 and len(coord_pct_suport_st) == 6:
        
        unghi_talpa_dr = calcul_unghi_orizont(coord_pct_suport_dr[0], coord_pct_suport_dr[1])
        unghi_talpa_st = 180 - calcul_unghi_orizont(coord_pct_suport_st[0], coord_pct_suport_st[1])

        
        if  40 <=  unghi_talpa_dr <= 75 and  40 <=  unghi_talpa_st <= 75:
            unghi_talpi = True
        else:
            indicatii.append("unghiul talpilor incorect")
    
        if coord_pct_suport_dr[2][0] < coord_pct_suport_dr[4][0] and coord_pct_suport_st[2][0] > coord_pct_suport_st[4][0]:
            pozitionare_picioare = True
        else:
            indicatii.append("Departati simetric picioarele!")
        
    
        if unghi_talpi and pozitionare_picioare:
            indicatii.append("Pozitie corecta! Ridicati mana dreapta pentru a incepe exercitiul")
            if coord_y_mana_dreapta is not None and coord_y_mana_dreapta < coord_pct_suport_dr[5][1] :
                if moment_mana_ridicata == None:
                    moment_mana_ridicata = time.time()
                    
                timp_mana_ridicata = time.time() - moment_mana_ridicata
                #print (timp_mana_ridicata)
                if timp_mana_ridicata >= 2:
                    indicatii.append("OK!")
                    moment_mana_ridicata = None
                    distanta_scalare = calcul_scalare_metrii_pixeli(coord_sold_dr_metri, coord_pct_suport_dr, coord_pct_suport_st)
                    stare = 1
                    

            
            
        #print (stare)
        #print(unghi_talpa_dr)
        #print(unghi_talpa_st)
        #print(abs(unghi_talpa_dr - unghi_talpa_st))

    else:
        indicatii.append("intrati in cadru")
    return indicatii

In [11]:
def calcul_scalare_metrii_pixeli(coord_sold_dr, coord_pct_suport_dr, coord_pct_suport_st):
    coord_x_mijloc = int(coord_pct_suport_dr[4][0] + coord_pct_suport_st[4][0])/2
    coord_y_mijloc = int(coord_pct_suport_dr[4][1] + coord_pct_suport_st[4][1])/2
    distanta_pixeli = math.sqrt((coord_x_mijloc -coord_pct_suport_dr[4][0]) ** 2 + (coord_y_mijloc - coord_pct_suport_dr[4][1]) ** 2)

    distanta_metri = math.sqrt( coord_sold_dr[0] ** 2 +  coord_sold_dr[1]** 2)
    scalare = distanta_metri / distanta_pixeli
    print(distanta_pixeli, "adaad")
    print(distanta_metri)
    return scalare
    # SA NU UIT sa schimb indexarea cu [4] ca arata ca dracu
    

In [12]:
def calcul_scalare_metrii_pixeli_profil(coord_sold_dr, coord_y_genunchi_m, coord_pct_suport_dr):
    #Presupun ca coordonatatele x si y ale sold_dr sunt aceleasi cu mijlocul intre solduri
    distanta_pixeli = math.sqrt((coord_pct_suport_dr[4][0] - coord_pct_suport_dr[3][0]) ** 2 + (coord_pct_suport_dr[4][1] - coord_pct_suport_dr[3][1]) ** 2)
    distanta_metri = math.sqrt(( coord_y_genunchi_m[0] - coord_sold_dr[0])** 2 +  (coord_y_genunchi_m[1] - coord_sold_dr[1])** 2)
    scalare = distanta_metri / distanta_pixeli

    return scalare

In [13]:
def pregatire_genuflexiune_profil(coord_pct_suport_dr, coord_pct_suport_st, coord_y_mana_dreapta, puncte):
    global stare
    global moment_mana_ridicata
    indicatii = ["Intorceti-va spre stanga pentru a initia urmatorul pas!"]

    print(adancime_coapse(puncte))
    print(len(coord_pct_suport_dr))
    print(len(coord_pct_suport_st))
    
    if len(coord_pct_suport_dr) == 6:
        if adancime_coapse(puncte) >= 0.175:
            indicatii = ["Ridicati mana dreapta pentru a incepe exercitiul"]
            if coord_y_mana_dreapta is not None and coord_y_mana_dreapta < coord_pct_suport_dr[5][1] :
                    if moment_mana_ridicata == None:
                        moment_mana_ridicata = time.time()
                                
                    timp_mana_ridicata = time.time() - moment_mana_ridicata
                    #print (timp_mana_ridicata)
                    if timp_mana_ridicata >= 2:
                        indicatii.append("OK!")
                        moment_mana_ridicata = None
                        stare = 3
    else:
        indicatii.append("intrati in cadru")

    return indicatii
        

In [14]:
def proces_genuflexiune_profil(coord_pct_suport_dr):
    global cnt_genuflexiuni_profil
    global stare
    global etapa_anterioara

    indicatii = [f"Counter: {cnt_genuflexiuni_profil}/5"]

    if len(coord_pct_suport_dr) == 6:
        flexie_genunchi_dr = calcul_unghi_3_puncte(coord_pct_suport_dr[2], coord_pct_suport_dr[3], coord_pct_suport_dr[4])

        if flexie_genunchi_dr >= 150:
            if etapa_anterioara == "urcare" and cnt_genuflexiuni_profil < 5:
                cnt_genuflexiuni_profil +=1
            if cnt_genuflexiuni_profil == 5:
                stare = 4
                etapa_anterioara = None
        
            etapa_anterioara = "coborare"
            etapa_genuflexiune = "coborare"

        elif flexie_genunchi_dr < 75:
                if etapa_anterioara == "coborare":
                    etapa_anterioara = "urcare"
                etapa_genuflexiune = "urcare"
    
        else:
            etapa_genuflexiune = etapa_anterioara

        try:
                indicatii.append(etapa_genuflexiune)
        except Exception as e:
                print(e)
    else:
        indicatii.append("intrati in cadru")
    
    return indicatii, etapa_genuflexiune
    
        

In [15]:
def stocare_date_front(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas,  coord_sold_dr_metrii, coord_sold_st_metrii, etapa_genuflexiune):
    dictionar_coordonate = {}
    
    if len(coord_pct_suport_st) == 6 and len(coord_pct_suport_dr) == 6:
        for (puncte_cheie_dr, puncte_cheie_st, coordonate_dr, coordonate_st) in zip(puncte_linii_suport_dr, puncte_linii_suport_st, coord_pct_suport_dr, coord_pct_suport_st) :
            dictionar_coordonate[puncte_cheie_st.name] = coordonate_st
            dictionar_coordonate[puncte_cheie_dr.name] = coordonate_dr
        dictionar_coordonate["NOSE"] = coord_y_nas
        dictionar_coordonate["RIGHT_HIP_m"] = coord_sold_dr_metrii
        dictionar_coordonate["LEFT_HIP_m"] = coord_sold_st_metrii
        
        coordonate_genuflexiuni_front[cnt_genuflexiuni_front][etapa_genuflexiune].append(dictionar_coordonate)
    else:
        coordonate_genuflexiuni_front[cnt_genuflexiuni_front][etapa_genuflexiune].append("frame_invalid")
        

In [16]:
def stocare_date_profil(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas, etapa_genuflexiune):
    dictionar_coordonate = {}

    if len(coord_pct_suport_dr) == 6:
        for puncte_cheie_dr, coordonate_dr in zip(puncte_linii_suport_dr, coord_pct_suport_dr):
            dictionar_coordonate[puncte_cheie_dr.name] = coordonate_dr

        dictionar_coordonate["NOSE"] = coord_y_nas
        coordonate_genuflexiuni_profil[cnt_genuflexiuni_profil][etapa_genuflexiune].append(dictionar_coordonate)
    else:
        coordonate_genuflexiuni_profil[cnt_genuflexiuni_profil][etapa_genuflexiune].append("frame_invalid")

In [17]:
def calcul_unghi_orizont(a, b):
    a = np.array(a)
    b = np.array(b)
    c = [0, 0.5, 0]
    d = [0.5, 0.5, 0]
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [18]:
def calcul_unghi_2_drepte(a, b, c, d):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    d = np.array(d)
    
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [19]:
def calcul_unghi_3_puncte(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    vect_dir_ab = np.subtract(b, a)
    vect_dir_bc = np.subtract(c, b)

    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_bc = np.linalg.norm(vect_dir_bc)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_bc) / (lungime_ab * lungime_bc))
    unghi = np.rad2deg(unghi)

    return 180 - unghi

In [20]:
def adancime_coapse(puncte):
    sold_stg = puncte[mp_pose.PoseLandmark.LEFT_HIP.value].z
    sold_drept = puncte[mp_pose.PoseLandmark.RIGHT_HIP.value].z

    
    return abs(sold_stg - sold_drept)

In [21]:
def calcul_hip_shift_front(coordonate_genuflexiuni_front, scalare):
    lista_hip_shift_frontal = []
    medii_hip_shift_frontal = [{"coborare": [], "urcare": []} for _ in range(5)]
    
    lista_diferenta_nivel_solduri = []
    medii_diferenta_nivel_solduri = [{"coborare": [], "urcare": []} for _ in range(5)]

    lista_diferenta_nivel_solduri_z = []
    medii_diferenta_nivel_solduri_z = [{"coborare": [], "urcare": []} for _ in range(5)]

    

    for i, repetitie in enumerate(coordonate_genuflexiuni_front):
        coord_x_mijloc_init = (repetitie["coborare"][0]["LEFT_HIP"][0] + repetitie["coborare"][0]["RIGHT_HIP"][0]) / 2
        for faza in ["coborare", "urcare"]:
            suma_hip_shift = 0
            suma_nivel = 0
            hip_shift_frontal = []
            diferenta_nivel_solduri = []
            diferenta_nivel_solduri_z = []
            
            for frame in repetitie[faza]:
                
                coord_x_mijloc = (frame["LEFT_HIP"][0] + frame["RIGHT_HIP"][0]) / 2
                hip_shift_frontal.append(round((coord_x_mijloc_init - coord_x_mijloc) * scalare * 100, 1))
                diferenta_nivel_solduri.append(round((frame["LEFT_HIP_m"][1] - frame["RIGHT_HIP_m"][1]) * 100, 1))
                diferenta_nivel_solduri_z.append(round((frame["LEFT_HIP_m"][2] - frame["RIGHT_HIP_m"][2]) * 100, 1))

            medii_hip_shift_frontal[i][faza] = round(sum(hip_shift_frontal) / len(hip_shift_frontal), 2)
            lista_hip_shift_frontal.append(hip_shift_frontal)

            medii_diferenta_nivel_solduri[i][faza] = round(sum(diferenta_nivel_solduri) / len(diferenta_nivel_solduri), 2)
            lista_diferenta_nivel_solduri.append(diferenta_nivel_solduri)

            medii_diferenta_nivel_solduri_z[i][faza] = round(sum(diferenta_nivel_solduri_z) / len(diferenta_nivel_solduri_z), 2)
            lista_diferenta_nivel_solduri_z.append(diferenta_nivel_solduri)


    print(medii_hip_shift_frontal)
    print(medii_diferenta_nivel_solduri)
    print(medii_diferenta_nivel_solduri_z)
    return medii_hip_shift_frontal, medii_diferenta_nivel_solduri, medii_diferenta_nivel_solduri_z

    

In [22]:
def calcul_adancime_genuflexiune(coordonate_genuflexiuni, scalare):
    #coordonate_genuflexiuni_front[cnt_genuflexiuni_front][etapa_genuflexiune].append(dictionar_coordonate)
    global puncte_metrii
    adancime_genuflexiune = []
    lista_adancime_genuflexiune = []
    
    for repetitie in coordonate_genuflexiuni:
        
        #print (repetitie["coborare"][0])
        if repetitie["coborare"]:
            coord_y_nas_init = repetitie["coborare"][0]["RIGHT_HIP"][1]
            coord_y_nas_final = repetitie["coborare"][-1]["RIGHT_HIP"][1]
            adancime_repetitie_pix = (coord_y_nas_final - coord_y_nas_init) #valoare in pixeli
        
        for frame in repetitie["coborare"]:
                coord_y_nas = frame["RIGHT_HIP"][1]
                adancime_pixeli = (coord_y_nas - coord_y_nas_init)
                adancime_repetitie_cm = round(adancime_repetitie_pix * scalare * 100, 1)
                lista_adancime_genuflexiune.append(adancime_repetitie_cm)
        if lista_adancime_genuflexiune:
            adancime_genuflexiune.append(lista_adancime_genuflexiune[-1])

    print (adancime_genuflexiune)
    return adancime_genuflexiune
        
    #daca nu sunt facute toate 5 genuflexiunile da eroare!!!!!
    #SA FAC  coord_pct_suport_st si dr DICTIONARE ca se simplifica mult chestiile mai ales la stocare date

In [23]:
def calcul_unghiuri_profil(coordonate_genuflexiuni_profil):
    lista_unghiuri = [[]]
    unghiuri_min = {"FLEXIE_SOLD": [], "FLEXIE_GENUNCHI": [], "UNGHI_TIBIE":[]}
    unghiuri_frame = {"FLEXIE_SOLD": [], "FLEXIE_GENUNCHI": [], "UNGHI_TIBIE":[]}

    for i, repetitie in enumerate(coordonate_genuflexiuni_profil):
        for faza in ["coborare", "urcare"]:
            for frame in repetitie[faza]:
                coord_umar = frame["RIGHT_SHOULDER"]
                coord_sold = frame["RIGHT_HIP"]
                coord_genunchi = frame["RIGHT_KNEE"]
                coord_glezna = frame["RIGHT_HEEL"]

                unghiuri_frame["FLEXIE_SOLD"].append(round(180 - calcul_unghi_3_puncte(coord_umar, coord_sold, coord_genunchi), 2))
                unghiuri_frame["FLEXIE_GENUNCHI"].append(round(180 - calcul_unghi_3_puncte(coord_sold, coord_genunchi, coord_glezna), 2))
                unghiuri_frame["UNGHI_TIBIE"].append(round(180 - calcul_unghi_orizont(coord_genunchi, coord_glezna), 2))
                lista_unghiuri.append(unghiuri_frame)
                
            if faza == "coborare" and unghiuri_frame["FLEXIE_SOLD"] and unghiuri_frame["FLEXIE_GENUNCHI"] and unghiuri_frame["UNGHI_TIBIE"]:
                unghiuri_min["FLEXIE_SOLD"].append(unghiuri_frame["FLEXIE_SOLD"][-1])
                unghiuri_min["FLEXIE_GENUNCHI"].append(unghiuri_frame["FLEXIE_GENUNCHI"][-1])
                unghiuri_min["UNGHI_TIBIE"].append(unghiuri_frame["UNGHI_TIBIE"][-1])

    print(unghiuri_min)
    return unghiuri_min
                

In [24]:
app = FastAPI()
prelucrare_stream()

In [25]:
@app.post("/date_extrase_genuflexiuni/")
async def post_date_extrase(date_extrase: List[dict]):
    #Daca am nevoie aici pot salva rezultatele intr-o baza de date
    try:
        with open("date_extrase_front.json", "w") as f:
            json.dump(date_extrase, f, indent=4)
    except Exception as e:
        print("Nu s-au putut trimite datele despre genuflexiuni de la API la pagina ", e)

In [26]:
@app.get("/date_extrase_genuflexiuni/")
async def get_date_extrase():
    try:
        with open("date_extrase_front.json", "r") as f:
            date_extrase = json.load(f)
        return date_extrase
    except Exception as e:
        print("Nu s-au putut primi datele despre genuflexiuni de la API la pagina ", e)

In [27]:
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

INFO:     Started server process [3608]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 55660) - "WebSocket /video_feed_live/" [accepted]
INFO:     connection open


0.9604355096817017
0.9974541068077087
0.046867553144693375
0.02056729607284069
0
0.958605945110321
0.9974388480186462
0.044663868844509125
0.019593041390180588
0
0.9547593593597412
0.9975031614303589
0.04478134959936142
0.020542796701192856
0
0.9473299384117126
0.9972336888313293
0.04275307059288025
0.01997029036283493
0
0.9428473711013794
0.997006356716156
0.04145224392414093
0.019508114084601402
0
0.9281472563743591
0.9969151020050049
0.03884686529636383
0.018330363556742668
0
0.9114640355110168
0.9968762993812561
0.03675367310643196
0.0177596565335989
0
0.9106076955795288
0.9969504475593567
0.035892657935619354
0.018253576010465622
0
0.9129762053489685
0.9969589114189148
0.036650676280260086
0.018755506724119186
0
0.9165186882019043
0.9969490170478821
0.03688245639204979
0.018653254956007004
0
0.9229552745819092
0.9970872402191162
0.03696649894118309
0.01852627471089363
0
0.9297455549240112
0.9972383975982666
0.04138921573758125
0.020335683599114418
0
0.9358903169631958
0.9972837567

Traceback (most recent call last):
  File "C:\Users\toatalumea\AppData\Local\Temp\ipykernel_3608\2518388818.py", line 54, in analiza_fandare_front
    stocare_date_front(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas, coord_sold_dr_metrii, coord_sold_st_metrii, etapa_genuflexiune)
  File "C:\Users\toatalumea\AppData\Local\Temp\ipykernel_3608\3360372456.py", line 12, in stocare_date_front
    coordonate_genuflexiuni_front[cnt_genuflexiuni_front][etapa_genuflexiune].append(dictionar_coordonate)
IndexError: list index out of range


0.9992645382881165
0.9999179840087891
0.9170842170715332
0.9359709620475769
0.052237510681152344
6
6
2
0.999148964881897
0.999886691570282
0.9024889469146729
0.917604923248291
0.06344960629940033
6
6
2
0.998966634273529
0.9998710751533508
0.8843908309936523
0.9033024907112122
0.05978848785161972
6
6
2
0.9989532828330994
0.9998809099197388
0.8864345550537109
0.9068672060966492
0.057251207530498505
6
6
2
0.9990096688270569
0.9998912215232849
0.8890292048454285
0.9115855097770691
0.061542337760329247
6
6
2
0.9990965723991394
0.9999013543128967
0.8976790904998779
0.9178231954574585
0.0764036737382412
6
6
2
0.9991855621337891
0.9999106526374817
0.9065693616867065
0.923820436000824
0.0913596861064434
6
6
2
0.9992663860321045
0.9999191164970398
0.9150572419166565
0.9290571212768555
0.10727515816688538
6
6
2
0.99933922290802
0.999926745891571
0.9229164719581604
0.9347140789031982
0.11997862532734871
6
6
2
0.9994040131568909
0.9999333620071411
0.92884361743927
0.9375119209289551
0.1354768201708

Traceback (most recent call last):
  File "C:\Users\toatalumea\AppData\Local\Temp\ipykernel_3608\2518388818.py", line 66, in analiza_fandare_front
    stocare_date_profil(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas, etapa_genuflexiune)
  File "C:\Users\toatalumea\AppData\Local\Temp\ipykernel_3608\2006731138.py", line 9, in stocare_date_profil
    coordonate_genuflexiuni_profil[cnt_genuflexiuni_profil][etapa_genuflexiune].append(dictionar_coordonate)
IndexError: list index out of range


...

0.9994873404502869
0.9997606873512268
0.8999972343444824
0.9025530219078064
0.017339838668704033
6
6
2
0.9994677901268005
0.9997522234916687
0.8805986642837524
0.8760024309158325
0.03030575066804886
6
6
2
0.99943608045578
0.999712347984314
0.8586793541908264
0.8487371206283569
0.03259650059044361
6
6
2
0.9993592500686646
0.9996657967567444
0.8425418734550476
0.827595055103302
0.03294369392096996
6
6
2
0.9993157386779785
0.9995688199996948
0.8200000524520874
0.7976419925689697
0.03470725566148758
6
6
2
0.9992620348930359
0.9994961619377136
0.8078029155731201
0.7774975895881653
0.03577069379389286
6
6
2
0.9991809129714966
0.9994977712631226
0.7962749004364014
0.7597522139549255
0.035133711993694305
6
6
2
0.9991578459739685
0.999518871307373
0.7905800938606262
0.7514041066169739
0.03438867814838886
6
6
2
0.9991283416748047
0.9995175004005432
0.7863714098930359
0.7402167320251465
0.04892861470580101
6
6
2
0.9989742040634155
0.9995160698890686
0.7786735892295837
0.7330094575881958
0.0691851

Traceback (most recent call last):
  File "C:\Users\toatalumea\AppData\Local\Temp\ipykernel_3608\2518388818.py", line 66, in analiza_fandare_front
    stocare_date_profil(coord_pct_suport_st, coord_pct_suport_dr, coord_y_nas, etapa_genuflexiune)
  File "C:\Users\toatalumea\AppData\Local\Temp\ipykernel_3608\2006731138.py", line 9, in stocare_date_profil
    coordonate_genuflexiuni_profil[cnt_genuflexiuni_profil][etapa_genuflexiune].append(dictionar_coordonate)
IndexError: list index out of range


[45.4, 35.9, 33.9, 35.4, 39.4]
[{'coborare': -0.51, 'urcare': 0.54}, {'coborare': -0.41, 'urcare': 0.36}, {'coborare': -0.21, 'urcare': 1.36}, {'coborare': -0.33, 'urcare': 0.59}, {'coborare': -0.39, 'urcare': 1.71}]
[{'coborare': 0.07, 'urcare': 0.29}, {'coborare': 0.11, 'urcare': 0.35}, {'coborare': -0.12, 'urcare': 0.16}, {'coborare': -0.06, 'urcare': -0.02}, {'coborare': -0.14, 'urcare': -0.02}]
[{'coborare': 4.12, 'urcare': 4.89}, {'coborare': 4.86, 'urcare': 4.94}, {'coborare': 5.06, 'urcare': 5.15}, {'coborare': 5.27, 'urcare': 5.09}, {'coborare': 5.13, 'urcare': 5.05}]
[38.9, 41.4, 35.9, 39.9, 37.9]
{'FLEXIE_SOLD': [73.96, 80.23, 77.08, 77.98, 80.08], 'FLEXIE_GENUNCHI': [100.22, 101.28, 96.19, 99.12, 99.87], 'UNGHI_TIBIE': [48.48, 48.61, 50.91, 49.09, 50.01]}
[{'adancime': 45.4, 'deviatie_coborare': -0.51, 'deviatie_urcare': 0.54, 'dif_nivel_solduri_coborare': 0.07, 'dif_nivel_solduri_urcare': 0.29, 'dif_nivel_solduri_z_coborare': 4.12, 'dif_nivel_solduri_z_urcare': 4.89}, {'ad

INFO:     connection closed


INFO:     127.0.0.1:56396 - "GET /date_extrase_genuflexiuni/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56398 - "POST /date_extrase_genuflexiuni/ HTTP/1.1" 200 OK


In [27]:
#Pentru salvare VID
#Feed-ul de la camera
vid_original = cv2.VideoCapture('video_test_4.mp4')
nr_frames = int(vid_original.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vid_original.get(cv2.CAP_PROP_FPS)

vid_prelucrat = cv2.VideoWriter('video_prelucrat.mp4',  cv2.VideoWriter_fourcc(*'MP4V'), fps, (int(vid_original.get(3)), int(vid_original.get(4)))) 

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    for frame_index in tqdm(range(nr_frames)):
        ret, frame = vid_original.read()

        #colorarea frame-ului din BGR in RGB
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #img.flags.writeable = False
        #detectarea modelului
        pose_model = pose.process(img)
        
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        #img = prelucrare_img(frame, 1)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        vid_prelucrat.write(img)
            
    vid_original.release()
    vid_prelucrat.release()

  0%|          | 0/904 [00:00<?, ?it/s]

In [25]:
for i, genuflexiune in enumerate(coordonate_genuflexiuni_front):
    print(f"\n🔹 Genuflexiunea {i+1}:")
    
    for etapa, frameuri in genuflexiune.items():
        print(f"  ➝ Etapa: {etapa}")
        
        for frame_idx, frame in enumerate(frameuri):
            print(f"    📌 Frame {frame_idx + 1}:")
            
            for landmark, coordonate in frame.items():
                print(f"      {landmark}: {coordonate}")


🔹 Genuflexiunea 1:
  ➝ Etapa: coborare
    📌 Frame 1:
      LEFT_FOOT_INDEX: (684, 636, 0.11514151096343994)
      RIGHT_FOOT_INDEX: (545, 633, 0.09928970038890839)
      LEFT_HEEL: (655, 604, 0.23105955123901367)
      RIGHT_HEEL: (570, 604, 0.21168041229248047)
      LEFT_ANKLE: (665, 594, 0.21984970569610596)
      RIGHT_ANKLE: (565, 594, 0.20005138218402863)
      LEFT_KNEE: (664, 498, 0.034484487026929855)
      RIGHT_KNEE: (570, 492, 0.03252072632312775)
      LEFT_HIP: (648, 376, 0.0009994176216423512)
      RIGHT_HIP: (586, 373, -0.0011825357796624303)
      LEFT_SHOULDER: (676, 212, -0.07479728758335114)
      RIGHT_SHOULDER: (570, 201, -0.07219967991113663)
      NOSE: (622, 154, -0.2583232820034027)
      RIGHT_HIP_m: (-0.09756495803594589, -0.000980400131084025, -0.019933834671974182)
      LEFT_HIP_m: (0.10019238293170929, -0.0030313192401081324, 0.023024093359708786)
    📌 Frame 2:
      LEFT_FOOT_INDEX: (684, 636, 0.11467911303043365)
      RIGHT_FOOT_INDEX: (545, 633, 

In [66]:
for i, genuflexiune in enumerate(coordonate_genuflexiuni_profil):
    print(f"\n🔹 Genuflexiunea {i+1}:")
    
    for etapa, frameuri in genuflexiune.items():
        print(f"  ➝ Etapa: {etapa}")
        
        for frame_idx, frame in enumerate(frameuri):
            print(f"    📌 Frame {frame_idx + 1}:")
            
            for landmark, coordonate in frame.items():
                print(f"      {landmark}: {coordonate}")


🔹 Genuflexiunea 1:
  ➝ Etapa: coborare
  ➝ Etapa: urcare

🔹 Genuflexiunea 2:
  ➝ Etapa: coborare
  ➝ Etapa: urcare

🔹 Genuflexiunea 3:
  ➝ Etapa: coborare
  ➝ Etapa: urcare

🔹 Genuflexiunea 4:
  ➝ Etapa: coborare
  ➝ Etapa: urcare

🔹 Genuflexiunea 5:
  ➝ Etapa: coborare
  ➝ Etapa: urcare


In [27]:
with open("genuflexiuni.json", "w") as f:
    json.dump(coordonate_genuflexiuni_profil, f, indent=4)

In [26]:
type(img.shape[0])

NameError: name 'img' is not defined

In [51]:
print(coordonate_genuflexiuni_front)

[{'coborare': [], 'urcare': []}, {'coborare': [], 'urcare': []}, {'coborare': [], 'urcare': []}, {'coborare': [], 'urcare': []}, {'coborare': [], 'urcare': []}]


In [26]:
print(cnt_genuflexiuni_front)

5


In [25]:
@app.websocket("/video_feed_live/")

SyntaxError: invalid syntax (761892867.py, line 1)